## Day 2: Red-Nosed Reports


Fortunately, the first location The Historians want to search isn't a long walk from the Chief Historian's office.

While the Red-Nosed Reindeer nuclear fusion/fission plant appears to contain no sign of the Chief Historian, the engineers there run up to you as soon as they see you. Apparently, they still talk about the time Rudolph was saved through molecular synthesis from a single electron.

They're quick to add that - since you're already here - they'd really appreciate your help analyzing some unusual data from the Red-Nosed reactor. You turn to check if The Historians are waiting for you, but they seem to have already divided into groups that are currently searching every corner of the facility. You offer to help with the unusual data.

The unusual data (your puzzle input) consists of many reports, one report per line. Each report is a list of numbers called levels that are separated by spaces. For example:

```
7 6 4 2 1
1 2 7 8 9
9 7 6 2 1
1 3 2 4 5
8 6 4 4 1
1 3 6 7 9
```

This example data contains six reports each containing five levels.

The engineers are trying to figure out which reports are safe. The Red-Nosed reactor safety systems can only tolerate levels that are either gradually increasing or gradually decreasing. So, a report only counts as safe if both of the following are true:

The levels are either all increasing or all decreasing.
Any two adjacent levels differ by at least one and at most three.
In the example above, the reports can be found safe or unsafe by checking those rules:

```
7 6 4 2 1: Safe because the levels are all decreasing by 1 or 2.
1 2 7 8 9: Unsafe because 2 7 is an increase of 5.
9 7 6 2 1: Unsafe because 6 2 is a decrease of 4.
1 3 2 4 5: Unsafe because 1 3 is increasing but 3 2 is decreasing.
8 6 4 4 1: Unsafe because 4 4 is neither an increase or a decrease.
1 3 6 7 9: Safe because the levels are all increasing by 1, 2, or 3.
```

So, in this example, 2 reports are safe.

Analyze the unusual data from the engineers. How many reports are safe?

In [1]:
import csv

In [2]:
def load_reports(filename):
    """
    Loads the reports from a file where each line is a space-separated list of integers.
    Uses Python's built-in csv module for robust parsing (handles spaces as delimiter).

    :param filename: The path to the file containing the reports (e.g., 'data/day2_data.txt').
    :return: List of lists, where each sublist is a report's levels (integers).
    """
    reports = []
    with open(filename, "r") as f:
        reader = csv.reader(f, delimiter=" ", skipinitialspace=True)
        for row in reader:
            if row:  # Skip empty rows
                try:
                    levels = [
                        int(x) for x in row if x.strip()
                    ]  # Filter out empty strings
                    if levels:  # Only add non-empty reports
                        reports.append(levels)
                except ValueError:
                    print(f"Skipping invalid row: {row}")
    return reports


# Example usage
reports = load_reports("../data/day2_data.txt")

# For verification (uncomment to use)
print(f"Number of reports: {len(reports)}")
# for i, report in enumerate(reports):
#     print(f"Report {i+1}: {report}")

Number of reports: 1000


In [ ]:
def count_safe_reports(reports, n_exception=0):
    """
    Counts the number of safe reports based on the rules:
    - Levels are either all increasing or all decreasing (no zeros or mixed directions).
    - Adjacent differences are between 1 and 3 inclusive in absolute value.

    :param reports: List of lists, where each sublist is a report's levels (integers).
    :return: Integer count of safe reports.
    """
    safe_count = 0
    for levels in reports:
        if len(levels) < 2:
            continue  # Too short to have diffs; assume unsafe

        diffs = [levels[i + 1] - levels[i] for i in range(len(levels) - 1)]

        # Check if all diffs are strictly positive or all strictly negative
        all_increasing = all(d > 0 for d in diffs)
        all_decreasing = all(d < 0 for d in diffs)

        if not (all_increasing or all_decreasing):
            continue  # Mixed or zeros

        # Check absolute diffs between 1 and 3 inclusive
        abs_diffs = [abs(d) for d in diffs]
        if min(abs_diffs) >= 1 and max(abs_diffs) <= 3:
            safe_count += 1

    return safe_count

In [4]:
count_safe_reports(load_reports("../data/day2_data.txt"))

490

In [5]:
def count_safe_reports_except(reports, n_exceptions):
    """
    Counts the number of safe reports based on the adjusted rules:
    - Levels are mostly increasing or mostly decreasing, with at most n_exceptions violations to the majority direction (violations include opposite direction and zeros).
    - There must be a clear majority direction (majority > violations).
    - All adjacent differences have |d| between 1 and 3 inclusive, but allow at most 1 exception (bad |d|, including zeros or |d|>3).

    :param reports: List of lists, where each sublist is a report's levels (integers).
    :param n_exceptions: Maximum allowed violations for the direction check.
    :return: Integer count of safe reports.
    """
    safe_count = 0
    for levels in reports:
        if len(levels) < 2:
            continue  # Too short to have diffs; assume unsafe

        diffs = [levels[i + 1] - levels[i] for i in range(len(levels) - 1)]

        # Count positives, negatives, zeros
        num_pos = sum(1 for d in diffs if d > 0)
        num_neg = sum(1 for d in diffs if d < 0)
        majority_count = max(num_pos, num_neg)
        violations = (
            len(diffs) - majority_count
        )  # Includes zeros and minority direction

        # Check for clear majority and not too many violations
        if violations > n_exceptions or majority_count <= violations:
            continue

        # Check abs diffs: allow at most 1 bad (|d| < 1 or |d| > 3, i.e., zeros or too large)
        num_bad_abs = sum(1 for d in diffs if not (1 <= abs(d) <= 3))
        if num_bad_abs > 1:
            continue

        safe_count += 1

    return safe_count

In [7]:
count_safe_reports_except(load_reports("../data/day2_data.txt"), n_exceptions=0)
# Not correct - should be 490

514

--- Part Two ---

The engineers are surprised by the low number of safe reports until they realize they forgot to tell you about the Problem Dampener.

The Problem Dampener is a reactor-mounted module that lets the reactor safety systems tolerate a single bad level in what would otherwise be a safe report. It's like the bad level never happened!

Now, the same rules apply as before, except if removing a single level from an unsafe report would make it safe, the report instead counts as safe.

More of the above example's reports are now safe:

```
7 6 4 2 1: Safe without removing any level.
1 2 7 8 9: Unsafe regardless of which level is removed.
9 7 6 2 1: Unsafe regardless of which level is removed.
1 3 2 4 5: Safe by removing the second level, 3.
8 6 4 4 1: Safe by removing the third level, 4.
1 3 6 7 9: Safe without removing any level.
```

Thanks to the Problem Dampener, 4 reports are actually safe!